In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
model_new = tf.keras.models.load_model("model.h5")

In [3]:
model_new.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input (InputLayer)          [(None, 11025, 1)]           0         []                            
                                                                                                  
 conv1d_1 (Conv1D)           (None, 11025, 16)            64        ['input[0][0]']               
                                                                                                  
 activation (Activation)     (None, 11025, 16)            0         ['conv1d_1[0][0]']            
                                                                                                  
 conv1d_2 (Conv1D)           (None, 11025, 16)            784       ['activation[0][0]']          
                                                                                              

 activation_11 (Activation)  (None, 689, 128)             0         ['conv1d_16[0][0]']           
                                                                                                  
 conv1d_17 (Conv1D)          (None, 689, 128)             49280     ['activation_11[0][0]']       
                                                                                                  
 conv1d_14 (Conv1D)          (None, 689, 128)             16512     ['max_pooling1d_3[0][0]']     
                                                                                                  
 add_4 (Add)                 (None, 689, 128)             0         ['conv1d_17[0][0]',           
                                                                     'conv1d_14[0][0]']           
                                                                                                  
 activation_12 (Activation)  (None, 689, 128)             0         ['add_4[0][0]']               
          

In [1]:
import pyaudio
import wave

In [2]:
FORMAT = pyaudio.paInt16 # Ses formatı
CHANNELS = 1 # Ses kanalı sayısı
RATE = 16000 # Örnekleme hızı (Hz)
CHUNK = 1024 # Her okumada alınacak ses parçası (bayt)
RECORD_SECONDS = 3000 # Kayıt süresi (saniye)
WAVE_FILE = "kayit.wav" # Kaydedilecek dosya adı

In [ ]:
# PyAudio nesnesi oluştur
audio = pyaudio.PyAudio()

# Ses akışını aç
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)

print("Ses kaydı başladı...")

# Ses verisini oku ve listeye ekle
frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("Ses kaydı bitti.")


# Ses akışını kapat
stream.stop_stream()
stream.close()
audio.terminate()

# Ses verisini wav dosyasına yaz
waveFile = wave.open(WAVE_FILE, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(frames))
waveFile.close()

Ses kaydı başladı...


In [7]:
import numpy as np
import librosa

In [8]:
np.__version__

'1.24.0'

In [14]:
# Ön işleme adımlarını uygula
y, sr = librosa.load(WAVE_FILE, sr=11025)
y = librosa.util.normalize(y)
mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)

# Veriyi girdi şekline dönüştür
mfccs = np.expand_dims(mfccs, axis=0)  # Örnek sayısı boyutunu ekleyin
mfccs = np.expand_dims(mfccs, axis=-1)  # Kanal boyutunu ekleyin

In [15]:
# Modelden çıktı al
preds = model_new.predict(mfccs)

# En yüksek olasılığa sahip sınıfı bul
class_index = np.argmax(preds[0])
class_names = ['jackson', 'nicolas', 'theo', 'yweweler'] # Sınıf isimleri (dökümana göre)
predicted_class = class_names[class_index]

# Sonucu ekrana yazdır
print("Tahmin edilen sınıf:", predicted_class)

ValueError: in user code:

    File "C:\Users\MUSTAFA\anaconda3\lib\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\MUSTAFA\anaconda3\lib\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\MUSTAFA\anaconda3\lib\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\MUSTAFA\anaconda3\lib\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "C:\Users\MUSTAFA\anaconda3\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\MUSTAFA\anaconda3\lib\site-packages\keras\src\engine\input_spec.py", line 298, in assert_input_compatibility
        ra